<a href="https://colab.research.google.com/github/benjaminnigjeh/keyProteoforms/blob/main/explainableAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update -qq
!apt-get install -y -qq mono-complete
!pip install -qq fisher-py

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Extracting templates from packages: 100%
Preconfiguring packages ...
Selecting previously unselected package mono-runtime-common.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .../0-mono-runtime-common_6.8.0.105+dfsg-3.2_amd64.deb ...
Unpacking mono-runtime-common (6.8.0.105+dfsg-3.2) ...
Selecting previously unselected package libmono-corlib4.5-dll.
Preparing to unpack .../1-libmono-corlib4.5-dll_6.8.0.105+dfsg-3.2_all.deb ...
Unpacking libmono-corlib4.5-dll (6.8.0.105+dfsg-3.2) ...
Selecting previously unselected package libmono-system-core4.0-cil.
Preparing to unpack .../2-libmono-system-core4.0-cil_6.8.0.105+dfsg-3.2_all.deb ...
Unpacking libmono-system-core4.0-cil (6.8.0.105+dfsg-3.2) ...
Selecting previously unselected package libmono-system-numeri

In [ ]:
from fisher_py.data.business import Scan
from fisher_py import RawFile

import re
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import subprocess

from scipy.signal import find_peaks
from scipy.signal import savgol_filter

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

from sklearn.datasets import make_classification
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler